In [1]:
from pyspark import SparkContext
from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1607406118605_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
glueContext = GlueContext(SparkContext.getOrCreate())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Connect to MongoDB Atlas data source
# Need to use legacy-style MongoDB connection string here (starts with mongodb:// instead of srv format)
spark = glueContext.spark_session.builder.appName("pyspark-atlas") \
    .appName("pyspark-atlas") \
    .config("spark.mongodb.input.uri", "mongodb://<username>:<password>@<mongodb_uri_in_legacy_format>") \
    .config("spark.mongodb.input.database", "glue-demo") \
    .config("spark.mongodb.input.collection", "restaurants") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Inspect Schema of the dataframe
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- address: struct (nullable = true)
 |    |-- street: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- zipcode: string (nullable = true)
 |-- cuisine: string (nullable = true)
 |-- name: string (nullable = true)
 |-- stars: double (nullable = true)

In [7]:
# Convert the dataframe into Glue DynamicFrame for further transformation
inputFrame = DynamicFrame.fromDF(df, glueContext, "restaurants")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Retain only the restaurant name and cuisine served from the source schema
nameOnly = inputFrame.select_fields(["name", "cuisine"]).toDF().distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Preview result
nameOnly.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+
|                name|      cuisine|
+--------------------+-------------+
|     Restaurant Iris|      Belgian|
|     Halls Chophouse|    Fast food|
|             V.Mertz|      Tibetan|
|              Forage|        Asian|
|The Tailor and th...|      Turkish|
|Fearrington House...|       Indian|
|Orchids at Palm C...|       Polish|
|La Mer at Halekulani|    Fast food|
|      Marx Bros Cafe|        Sushi|
|              Forage|       German|
|   The Flatiron Cafe|       Polish|
|Michaels - South ...|        Sushi|
|              Herons|        Irish|
|                NAOE|     Moroccan|
|    The Painted Lady|     Barbecue|
|    The Painted Lady|Mediterranean|
|Rudy & Paco Resta...|    Caribbean|
|              n/naka|       French|
|             ShinBay|        Greek|
|Camp Verde Genera...|        Asian|
+--------------------+-------------+
only showing top 20 rows

In [10]:
# Convert the dataframe back into Glue DynamicFrame before saving the result into S3
outputFrame = DynamicFrame.fromDF(nameOnly, glueContext, "restaurantsAndCuisine")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Now we save the result into S3 and also in parquet format, facilitating querying by Athena
glueContext.write_dynamic_frame.from_options(
    frame = outputFrame,
    connection_type = "s3",
    connection_options = {"path": "s3://path-to-output"},
    format = "parquet"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…